In [2]:
project_name="mice"

In [1]:
import os

In [3]:
%pwd

'f:\\ML_Ops_learning\\Mice-Protein-Expression\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'f:\\ML_Ops_learning\\Mice-Protein-Expression'

## config/config.yaml

In [ ]:
%%writefile -a config/config.yaml

model_trainer:
  root_dir: artifacts/model_trainer
  train_data_path: artifacts/data_transformation/train.csv
  test_data_path: artifacts/data_transformation/test.csv
  model_name: model.joblib

# Entity Folder
1. Goto entity folder  src-> wine_quality -> entity-> __init__.py 
2. paste below code in __init__.py constructor file

In [18]:
project_name

'mice'

In [19]:
from dataclasses import dataclass
from pathlib import Path

In [34]:
#%%writefile -a src/{project_name}/entity/__init__.py


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

# Config folder

1. Copy entire ConfigurationManager class
2. Goto config folder src->wine_quality->config->configuration.py and paste entire ConfigurationManager class in configuration.py
3. Import DataIngestionConfig from wine_quality.entity using this line  
from wine_quality.entity import DataIngestionConfig

In [20]:
project_name

'mice'

In [7]:
from mice.constants import *
from mice.utils import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [22]:
# %%writefile -a src/{project_name}/config/__init__.py   



#     def get_model_trainer_config(self) -> ModelTrainerConfig:
#         config = self.config.model_trainer
#         params = self.params.ElasticNet
#         schema =  self.schema.TARGET_COLUMN

#         create_directories([config.root_dir])

#         model_trainer_config = ModelTrainerConfig(
#             root_dir=config.root_dir,
#             train_data_path = config.train_data_path,
#             test_data_path = config.test_data_path,
#             model_name = config.model_name,
#             alpha = params.alpha,
#             l1_ratio = params.l1_ratio,
#             target_column = schema.name
            
#         )

#         return model_trainer_config

# Component folder

1. Copy entire DataIngestion class
2. Goto Component folder [ src-> wine_quality -> components - > data_ingestion.py ]
2. Paste DataIngestionclass in data_ingestion.py.py
3. Import dataIngestionconfig from entity using this line 

from projectname.entity import dataIngestionconfig

In [23]:
project_name 

'mice'

In [25]:
# %%writefile -a src/{project_name}/components/model_trainer.py
# from mice.entity import ModelTrainerConfig

In [36]:
#%%writefile -a src/{project_name}/components/model_trainer.py
import pandas as pd
import os
from mice import logger
from sklearn.linear_model import ElasticNet
import joblib

In [28]:
#%%writefile -a src/{project_name}/components/model_trainer.py

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column,'MouseID'], axis=1)
        test_x = test_data.drop([self.config.target_column,'MouseID'], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))



In [17]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-11-30 21:08:54,923] - mice - INFO - __init__ file - LineNum:32   - yaml file: config\config.yaml loaded successfully
[2023-11-30 21:08:54,927] - mice - INFO - __init__ file - LineNum:32   - yaml file: params.yaml loaded successfully
[2023-11-30 21:08:54,951] - mice - INFO - __init__ file - LineNum:32   - yaml file: schema.yaml loaded successfully
[2023-11-30 21:08:54,954] - mice - INFO - __init__ file - LineNum:52   - created directory at: artifacts
[2023-11-30 21:08:54,957] - mice - INFO - __init__ file - LineNum:52   - created directory at: artifacts/model_trainer


## Into the file
pipeline/stage_04_model_trainer.py

In [29]:
#%%writefile src/{project_name}/pipeline/stage_04_model_trainer.py
from mice.config import ConfigurationManager
from mice.components.model_trainer import ModelTrainer



class ModelTrainerTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        model_trainer_config = config.get_model_trainer_config()
        model_trainer_config = ModelTrainer(config=model_trainer_config)
        model_trainer_config.train()

Overwriting src/mice/pipeline/stage_04_model_trainer.py


Main


In [30]:
#%%writefile -a main.py

from mice.pipeline.stage_04_model_trainer import ModelTrainerTrainingPipeline
STAGE_NAME = "Model Trainer stage"
try: 
   logger.info(f"*******************")
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
   model_trainer = ModelTrainerTrainingPipeline()
   model_trainer.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
        logger.exception(e)
        raise e

Appending to main.py


In [35]:
os.system("python main.py")

0